# EDA

1. Скачайте датасет fifаs2.сsv. Проанализируйте его и определите, какие данные являются неполными. Удалите
ненужные колонки и недостающие значения.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

In [2]:
df = spark.read.option('header', True).csv('fifa_s2.csv')

In [3]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [4]:
df.show(5)

+-----+---------+---+-----------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|   ID|     Name|Age|Nationality|Overall|Potential|               Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|
+-----+---------+---+-----------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
| 1179|G. Buffon| 40|      Italy|     88|       88|Paris Saint-Germain| 4000|  77|         Right|                       4|          1|      GK|  2018|          01/01/2019|6.333333333|   203|          7400|
| 5479| Casillas| 37|      Spain|     82|       82|           FC Porto| 1500|  10|          Left|                       4|          1|    NULL|  2015|          01/01/2019|6.083

Анализ пропусков

In [5]:
from pyspark.sql.functions import count, isnan, col, when
df_Columns = df.columns
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df_Columns]).show()

+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+
| ID|Name|Age|Nationality|Overall|Potential|Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|Height|Weight|Release Clause|
+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+
|  0|   0|  0|          7|      0|        0| 241|  257|   0|             0|                      10|          6|      21|     0|                 273|     0|     0|             3|
+---+----+---+-----------+-------+---------+----+-----+----+--------------+------------------------+-----------+--------+------+--------------------+------+------+--------------+



In [6]:
for cols in df_Columns:
    print(f'в колонке {cols} {df.filter(col(cols).isNull()).count()} пропусков, или {round(df.filter(col(cols).isNull()).count()/df.count()*100, 2)} %')

в колонке ID 0 пропусков, или 0.0 %
в колонке Name 0 пропусков, или 0.0 %
в колонке Age 0 пропусков, или 0.0 %
в колонке Nationality 7 пропусков, или 0.29 %
в колонке Overall 0 пропусков, или 0.0 %
в колонке Potential 0 пропусков, или 0.0 %
в колонке Club 241 пропусков, или 10.05 %
в колонке Value 257 пропусков, или 10.71 %
в колонке Wage 0 пропусков, или 0.0 %
в колонке Preferred Foot 0 пропусков, или 0.0 %
в колонке International Reputation 10 пропусков, или 0.42 %
в колонке Skill Moves 6 пропусков, или 0.25 %
в колонке Position 21 пропусков, или 0.88 %
в колонке Joined 0 пропусков, или 0.0 %
в колонке Contract Valid Until 273 пропусков, или 11.38 %
в колонке Height 0 пропусков, или 0.0 %
в колонке Weight 0 пропусков, или 0.0 %
в колонке Release Clause 3 пропусков, или 0.13 %


In [7]:
df.toPandas().isna().mean() * 100

ID                           0.000000
Name                         0.000000
Age                          0.000000
Nationality                  0.291788
Overall                      0.000000
Potential                    0.000000
Club                        10.045852
Value                       10.712797
Wage                         0.000000
Preferred Foot               0.000000
International Reputation     0.416840
Skill Moves                  0.250104
Position                     0.875365
Joined                       0.000000
Contract Valid Until        11.379742
Height                       0.000000
Weight                       0.000000
Release Clause               0.125052
dtype: float64

In [26]:
# Смотрим типы, чтобы понимать чем заполнять пропуски, строками или числами
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- Potential: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: string (nullable = true)
 |-- Skill Moves: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Contract Valid Until: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Release Clause: string (nullable = true)



In [27]:
# Всё заполним константными строками. Переводить в числа пока не будем
df = df.fillna(value="unknown")
df.toPandas().isna().mean() * 100

ID                          0.0
Name                        0.0
Age                         0.0
Nationality                 0.0
Overall                     0.0
Potential                   0.0
Club                        0.0
Value                       0.0
Wage                        0.0
Preferred Foot              0.0
International Reputation    0.0
Skill Moves                 0.0
Position                    0.0
Joined                      0.0
Contract Valid Until        0.0
Height                      0.0
Weight                      0.0
Release Clause              0.0
dtype: float64

In [25]:
# Колонки Overall и Potential похожи, проверим сколько различных значений
df.filter(df['Overall'] != df['Potential']).count()

1576

Удалять колонки не будем

# Дубликаты

2. Найдите в датафрейме полные дубликаты и удалите их. Значения могут быть одинаковыми, но написаны по-разному. Например, может отличаться размер регистра (заглавные и строчные буквы). Особое внимание уделить колонке с названиями команд.

In [14]:
from pyspark.sql.functions import lower

for col in df.columns:
    df = df.withColumn(col, lower(col))
df.show()

+-----+-----------------+---+-------------------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|   ID|             Name|Age|        Nationality|Overall|Potential|               Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|
+-----+-----------------+---+-------------------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
| 1179|        g. buffon| 40|              italy|     88|       88|paris saint-germain| 4000|  77|         right|                       4|          1|      gk|  2018|          01/01/2019|6.333333333|   203|          7400|
| 5479|         casillas| 37|              spain|     82|       82|           fc porto| 1500|  10|          left

In [22]:
# Количество дублей
df.count() - df.distinct().count()

2

In [17]:
df \
    .groupby(df.columns) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show()

+------+----------+---+-----------+-------+---------+------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+-----+
|    ID|      Name|Age|Nationality|Overall|Potential|        Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|count|
+------+----------+---+-----------+-------+---------+------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+-----+
|181872|  a. vidal| 31|      chile|     85|       85|fc barcelona|26000| 205|         right|                       4|          3|     cam|  2018|          01/01/2021|5.916666667|   165|         53300|    2|
|168651|i. rakitić| 30|    croatia|     87|       87|fc barcelona|46500| 260|         right|                       4|          3|     rcm|  2014|          01/01/2021|      

In [20]:
df.filter((df['id'] == 181872) | (df['id'] == 168651)).show()

+------+----------+---+-----------+-------+---------+------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|    ID|      Name|Age|Nationality|Overall|Potential|        Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|
+------+----------+---+-----------+-------+---------+------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|168651|i. rakitić| 30|    croatia|     87|       87|fc barcelona|46500| 260|         right|                       4|          3|     rcm|  2014|          01/01/2021|          6|   172|         95300|
|168651|i. rakitić| 30|    croatia|     87|       87|fc barcelona|46500| 260|         right|                       4|          3|     rcm|  2014|          01/01/2021|          6|   172|         95

In [23]:
df = df.dropDuplicates()
df.count()

2397

# Функция

3. Напишите функцию, которая добавит колонку с разбиением возраста по группам: до 20, от 20 до 30, от 30 до
36 и старше 36. Посчитайте количество футболистов в каждой категории.

In [31]:
from pyspark.sql.types import IntegerType

df = df.withColumn('Age', df['Age'].cast(IntegerType()))
df.printSchema()

root
 |-- ID: string (nullable = false)
 |-- Name: string (nullable = false)
 |-- Age: integer (nullable = true)
 |-- Nationality: string (nullable = false)
 |-- Overall: string (nullable = false)
 |-- Potential: string (nullable = false)
 |-- Club: string (nullable = false)
 |-- Value: string (nullable = false)
 |-- Wage: string (nullable = false)
 |-- Preferred Foot: string (nullable = false)
 |-- International Reputation: string (nullable = false)
 |-- Skill Moves: string (nullable = false)
 |-- Position: string (nullable = false)
 |-- Joined: string (nullable = false)
 |-- Contract Valid Until: string (nullable = false)
 |-- Height: string (nullable = false)
 |-- Weight: string (nullable = false)
 |-- Release Clause: string (nullable = false)



In [46]:
from pyspark.sql.functions import lit

def age_category(df):
    df_2 = df.filter(df.Age < 20).withColumn('Age_category', lit('< 20'))
    df_2 = df_2.union(df.filter((df.Age >= 20) & (df.Age < 30)).withColumn('Age_category', lit('20-30')))
    df_2 = df_2.union(df.filter((df.Age >= 30) & (df.Age < 36)).withColumn('Age_category', lit('30-36')))
    df_2 = df_2.union(df.filter(df.Age >= 36).withColumn('Age_category', lit('> 36')))
    return df_2
        

In [49]:
df_2 = age_category(df)
df_2.filter('Age == 25').show(10)

+------+-------------+---+-----------+-------+---------+--------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+------------+
|    ID|         Name|Age|Nationality|Overall|Potential|                Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|Age_category|
+------+-------------+---+-----------+-------+---------+--------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+------------+
|203502|      b. reid| 25|    england|     73|       78|        cardiff city| 5500|  37|         right|                       1|          3|      st|  2018|          01/01/2022|5.583333333|   150|         11400|       20-30|
|206159|   s. sabelli| 25|      italy|     68|       71|             brescia|  925|   2|         rig

In [50]:
df_2.filter('Age == 32').show(10)

+------+---------------+---+-----------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+------------+
|    ID|           Name|Age|Nationality|Overall|Potential|               Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|Age_category|
+------+---------------+---+-----------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+------------+
|164468|      g. cahill| 32|    england|     80|       80|            chelsea| 8500|  99|         right|                       3|          2|      cb|  2012|          01/01/2019|6.333333333|   194|         15700|       30-36|
|193369|          galán| 32|      spain|     69|       69|cf rayo majadahonda|  575|   4|       

In [51]:
df_2.filter('Age == 37').show(10)

+------+-------------+---+-----------+-------+---------+--------------------+-------+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+------------+
|    ID|         Name|Age|Nationality|Overall|Potential|                Club|  Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|Age_category|
+------+-------------+---+-----------+-------+---------+--------------------+-------+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+------------+
|145047| bruno saltor| 37|      spain|     74|       74|brighton & hove a...|    650|  25|         right|                       1|          3|      rb|  2012|          01/01/2019|5.916666667|   168|          1200|        > 36|
| 54008|   s. bywater| 37|    england|     63|       63|       burton albion|     20|   1|  

In [54]:
df_2.groupBy('Age_category').count().show()

+------------+-----+
|Age_category|count|
+------------+-----+
|        < 20|  270|
|       20-30| 1646|
|       30-36|  458|
|        > 36|   23|
+------------+-----+

